# Esercizio 7

Prendere in input un file in formato `FASTQ` e determinare:

- la distribuzione della qualità media per read
- la distribuzione della qualità media per posizione
- la distribuzione delle basi A, C, G e T per posizione

Eseguire inoltre il *trimming* dei reads scegliendo una soglia minima di qualità, sulla base dei risultati ottenuti sopra e scartare i reads che risultano troppo corti.

---

## Dataset in input

In [ ]:
fastq_file_name = './SRR18961685-5000.fastq'

## Definizione della funzione `ascii_to_quality()`

La funzione prende come argomento un carattere e restituisce il valore di qualità codificato secondo la seguente funzione:

    C = ASCII(min(93,Q)+33)

In [ ]:
def ascii_to_quality(c):
    return ord(c)-33

## Lettura del dataset `FASTQ`

a) Leggere il file `FASTQ` in input.

In [ ]:
with open(fastq_file_name, 'r') as fastq_input_file:
    input_file_rows = [row.rstrip() for row in fastq_input_file.readlines()]

In [ ]:
input_file_rows

b) Raggruppare i *record* a gruppi di quattro (cioé per read in formato `FASTQ`).

In [ ]:
fastq_read_list = [input_file_rows[i:i+4] for i in range(0, len(input_file_rows), 4)]

In [ ]:
#fastq_read_list

c) Recuperare il numero di reads e la loro lunghezza (supposta costante).

In [ ]:
read_length = len(fastq_read_list[0][1])

In [ ]:
read_length

In [ ]:
number_of_reads = len(fastq_read_list)

In [ ]:
number_of_reads

## Determinare la distribuzione della qualità media per read

a) Creare la matrice dei valori di qualità dei reads, in modo tale che l'i-esima riga contenga le qualità delle basi dell'i-esimo read.

In [ ]:
import numpy as np

In [ ]:
qualities_per_read = np.array([list(map(ascii_to_quality, list(read[3]))) for read in fastq_read_list])

In [ ]:
qualities_per_read

b) Ottenere la lista delle qualità medie dei reads, arrotondate all'intero più vicino.

In [ ]:
mean_quality_per_read = [round(qualities.mean()) for qualities in qualities_per_read]

In [ ]:
mean_quality_per_read

c) Determinare la distribuzione delle frequenze assolute delle qualità medie dei reads.

In [ ]:
(unique_values, counts) = np.unique(mean_quality_per_read, return_counts = True)

In [ ]:
unique_values

In [ ]:
counts

d) Disegnare il diagramma a barre delle frequenze assolute.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
f = plt.figure(figsize = (10,6))
plt.bar(unique_values, counts, color = 'green', width = 0.5)
plt.ylabel('#reads')
plt.xlabel('Phred values')
plt.title('Mean quality per read')
plt.show()

## Determinare la distribuzione della qualità media per posizione

a) Ottenere la matrice delle qualità per posizione, in modo tale che l'i-esima riga contenga le qualità dell'i-esima base dei reads.

In [ ]:
qualities_per_pos = np.transpose(qualities_per_read)

In [ ]:
qualities_per_pos

b) Ottenere la lista delle qualità medie per posizione, arrotondate all'intero più vicino.

In [ ]:
mean_quality_per_pos = [round(qualities.mean()) for qualities in qualities_per_pos]

In [ ]:
mean_quality_per_pos

c) Disegnare il diagramma delle qualità medie per posizione.

In [ ]:
f = plt.figure(figsize = (10,6))
plt.plot(mean_quality_per_pos)
plt.ylabel('Phred values')
plt.xlabel('Positions')
plt.title('Mean quality per position')
plt.show()

## Determinare la distribuzione delle basi `A`, `C`, `G`, `T` per posizione

a) Creare la matrice delle basi dei reads, in modo tale che l'i-esima riga contenga le i-esime basi dei reads.

In [ ]:
bases_per_pos = np.array([list(read[1]) for read in fastq_read_list]).transpose()

In [ ]:
bases_per_pos

b) Per ogni posizione, ottenere la distribuzione delle frequenze assolute delle basi.

In [ ]:
count_list = [np.unique(bases, return_counts = True) for bases in bases_per_pos]

In [ ]:
count_list

c) Per ogni posizione, disegnare il diagramma a barre della distribuzione delle basi.

In [ ]:
for (i, counts) in enumerate(count_list):
    f = plt.figure(figsize = (5,3))
    plt.bar(counts[0], counts[1], color = 'green', width = 0.4)
    plt.ylabel('#positions')
    plt.title('Base distribution at position ' + str(i))
    plt.show()

## Eseguire il *trimming* dei reads

a) Scegliere una soglia minima di qualità.

In [ ]:
min_quality = 20

**b) Definire una funzione che prenda come argomento una stringa di qualità e una soglia minima e restituisca l'intervallo di *trimming* (cioé, il più lungo intervallo di posizioni contenente solo caratteri che codificano una qualità almeno uguale alla soglia minima).**

In [ ]:
def get_trimming_interval(quality_string, min_quality):
    bool_list = [ascii_to_quality(c) >= min_quality for c in quality_string]
    start_list = [i for (i,b) in enumerate(bool_list, 0) if b == True and (i == 0 or bool_list[i-1] == False)]
    end_list = [i for (i,b) in enumerate(bool_list, 0) if b == True and (i == len(bool_list)-1 or bool_list[i+1] == False)]
    start_list[:0] = [1]
    end_list[:0] = [0]
    interval_lengths = list((np.array(end_list) - np.array(start_list) + 1))
    max_index = interval_lengths.index(max(interval_lengths))
    return (start_list[max_index], end_list[max_index]+1)

---
**SPIEGAZIONE**

Stringa di qualità di prova.

In [ ]:
quality_string = '@A?ACDBECEEECECEEECECEE>@@CEEECEC?EEC?CEE//EC0EE//:0E.//C-@=+D:0..C)D-?///:1@@@>ABCCDBCDDDECDDDDDCDDDDECDDDDECDDDDECD<D,DCDD+DEC,;D,6CD<,*.C,-:,BC,-@-,@'

1) Costruire una lista di valori booleani, in modo tale che l'i-esimo valore è `True` se l'i-esimo carattere codifica una qualità maggiore o uguale alla soglia minima.

**NB**: Le posizioni di inizio e fine della più lunga sottolista di soli valori `True` saranno le posizioni di inizio e fine della sottostringa di read che ha qualità almeno pari alla soglia minima.

In [ ]:
bool_list = [ascii_to_quality(c) >= min_quality for c in quality_string]

In [ ]:
bool_list

2. Determinare la lista delle sole posizioni che contengono un valore `True` preceduto da un valore `False`. La posizione `0` è da intendere preceduta da un valore `False`.

In [ ]:
start_list = [i for (i,b) in enumerate(bool_list, 0) if b == True and (i == 0 or bool_list[i-1] == False)]

In [ ]:
start_list

3. Aggiungere in testa la posizione di *default* `1`.

In [ ]:
start_list[:0] = [1]

In [ ]:
start_list

4. Ottenere la lista delle sole posizioni che contengono un valore `True` seguito da un valore `False`. L'ultima posizione è da intendere seguita da un valore `False`.

In [ ]:
end_list = [i for (i,b) in enumerate(bool_list, 0) if b == True and (i == len(bool_list)-1 or bool_list[i+1] == False)]

In [ ]:
end_list

5. Aggiungere in testa la posizione di *default* `0`.

In [ ]:
end_list[:0] = [0]

In [ ]:
end_list

6. Ottenere la lista delle lunghezze degli intervalli di soli valori `True`.

In [ ]:
interval_lengths = list((np.array(end_list) - np.array(start_list) + 1))

In [ ]:
interval_lengths

7. Ottenere la posizione che contiene la massima lunghezza.

In [ ]:
max_index = interval_lengths.index(max(interval_lengths))

In [ ]:
max_index

8. Determinare le posizioni di inizio e fine dell'intervallo di massima lunghezza.

In [ ]:
(interval_start, interval_end) = (start_list[max_index], end_list[max_index])

In [ ]:
interval_start

In [ ]:
interval_end

`[interval_start, interval_end+1]` è l'intervallo di *trimming*.

**NB**: se viene restituita la lista `[1,1]` allora significa che tutti i valori in `bool_list` sono uguali a `False` (cioé tutte le qualità sono al di sotto della soglia minima), e il *trimming* restituirà la stringa vuota.

---

**c) Definire una funzione che prenda come argomenti una soglia minima di qualità e un read in formato `FASTQ` (lista dei quattro *record* `FASTQ`) e restituisca il read in formato `FASTQ` dopo essere stato sottoposto a *trimming*.**

In [ ]:
def get_trimmed_read(min_quality, fastq_read):
    (trimming_start, trimming_end) = get_trimming_interval(fastq_read[3], min_quality)
    trimmed_read = fastq_read[1][trimming_start:trimming_end]
    trimmed_quality_string = fastq_read[3][trimming_start:trimming_end]
    return [fastq_read[0], trimmed_read, fastq_read[2], trimmed_quality_string]

**d) Effettuare il *trimming* dei reads.**

In [ ]:
fastq_read_list = [get_trimmed_read(min_quality, read) for read in fastq_read_list]

In [ ]:
fastq_read_list

**e) Eliminare i reads troppo corti.**

In [ ]:
min_length = 30

In [ ]:
fastq_read_list = [read for read in fastq_read_list if len(read[1]) >= min_length]

In [ ]:
fastq_read_list

## Determinare la distribuzione delle qualità medie dei reads dopo il *trimming*

a) Ottenere la lista dei valori di qualità dei reads.

In [ ]:
qualities_per_read = [list(map(ascii_to_quality, list(read[3]))) for read in fastq_read_list]

In [ ]:
qualities_per_read

b) Ottenere la lista delle qualità medie dei reads, arrotondate all'intero più vicino.

In [ ]:
import statistics

In [ ]:
mean_quality_per_read = [round(statistics.mean(qualities)) for qualities in qualities_per_read]

In [ ]:
mean_quality_per_read

c) Determinare la distribuzione delle frequenze assolute delle qualità medie dei reads.

In [ ]:
(unique_values, counts) = np.unique(mean_quality_per_read, return_counts = True)

In [ ]:
unique_values

In [ ]:
counts

d) Disegnare il diagramma a barre

In [ ]:
f = plt.figure(figsize = (10,6))
plt.bar(unique_values, counts, color = 'green', width = 0.5)
plt.ylabel('#reads')
plt.xlabel('Phred values')
plt.title('Mean quality per read')
plt.show()